# IMPORT DES LIBRAIRIES

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline

# OPTIONS PANDAS 

In [2]:
pd.options.display.float_format = '{:.0f}'.format

# OUVERTURE DE NOTRE DATASET

In [3]:
df = pd.read_csv('../DATA/OUT/new_dataset_hors_outliers_surfaces_and_piece.csv')
df.head()

,IDVentes,Prix,Date,SurfaceTerrain,SurfaceBati,SurfaceCarrez,NombreLots,NombrePiecesPrincipales,CodePostal,TypeLocalName,slice,quarter,Year,Month,AvecTerrain,SurfaceBatiCat,ID,PrixTerrainM2,PrixBatiM2,PrixCarrezM2
0,174617,103950,2021-01-06,0,19,19,1,1,33600,Appartement,"[100000.0, 150000.0)",1,2021,1,False,"(0.999, 35.0]",1,0,5471,5468
1,174618,214380,2021-01-05,0,38,39,2,1,33000,Appartement,"[200000.0, 250000.0)",1,2021,1,False,"(35.0, 47.0]",2,0,5642,5558
2,174619,177000,2021-01-08,0,63,0,3,1,33320,Appartement,"[150000.0, 200000.0)",1,2021,1,False,"(60.0, 70.0]",3,0,2810,0
3,174620,153000,2021-01-08,0,30,0,2,1,33800,Appartement,"[150000.0, 200000.0)",1,2021,1,False,"(0.999, 35.0]",4,0,5100,0
4,174621,223170,2021-01-08,0,61,0,3,1,33127,Appartement,"[200000.0, 250000.0)",1,2021,1,False,"(60.0, 70.0]",5,0,3659,0


# SUPPRESSION DES FEATURES NON ESSENTIELLES

In [4]:
# Préparation de df_model en excluant les colonnes spécifiées dans votre code initial
columns_to_drop = ['Date', 'IDVentes', 'slice', 'quarter', 'Year', 'Month', 'AvecTerrain', 'SurfaceBatiCat', 'ID', 'PrixTerrainM2', 'PrixBatiM2', 'PrixCarrezM2']

# Créer df_model
df_model = df.drop(columns=columns_to_drop)

# Vérifier s'il reste des colonnes non numériques qui doivent être exclues ou traitées séparément
# df_model.select_dtypes(exclude=['number']).columns.tolist()

df_model = df_model.dropna(subset=['Prix', 'SurfaceBati', 'CodePostal'])
df_model.isna().sum()

Prix                       0
SurfaceTerrain             0
SurfaceBati                0
SurfaceCarrez              0
NombreLots                 0
NombrePiecesPrincipales    0
CodePostal                 0
TypeLocalName              0
dtype: int64

In [5]:
df_model

,Prix,SurfaceTerrain,SurfaceBati,SurfaceCarrez,NombreLots,NombrePiecesPrincipales,CodePostal,TypeLocalName
0,103950,0,19,19,1,1,33600,Appartement
1,214380,0,38,39,2,1,33000,Appartement
2,177000,0,63,0,3,1,33320,Appartement
3,153000,0,30,0,2,1,33800,Appartement
4,223170,0,61,0,3,1,33127,Appartement
...,...,...,...,...,...,...,...,...
147835,140000,1922,200,0,8,0,33860,Maison
147836,80000,237,73,0,4,0,33820,Maison
147837,155000,600,78,0,4,0,33830,Maison
147838,111150,51,82,0,2,0,33210,Maison


# TRANSFORMATION DE LOCALISATION EN NUMERIQUE (ONE HOT ENCODER)

In [7]:
# Réinitialiser l'index pour s'assurer que l'alignement est conservé après la suppression des NaN
df_model.reset_index(drop=True, inplace=True)

# Préparer la colonne Localisation pour le One-Hot Encoding
localisation = df_model[['CodePostal', 'TypeLocalName']]

# Instancier le OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)
localisation_encoded = encoder.fit_transform(localisation)

# Convertir les résultats encodés en DataFrame
localisation_encoded_df = pd.DataFrame(localisation_encoded,
                                       columns=encoder.get_feature_names_out(['CodePostal', 'TypeLocalName']))

# Réinitialiser l'index sur le DataFrame encodé pour garantir l'alignement lors de la fusion
localisation_encoded_df.reset_index(drop=True, inplace=True)

# Fusionner le DataFrame encodé avec le df original en s'assurant que les indices sont alignés
df_model = pd.concat([df_model, localisation_encoded_df], axis=1)

# Supprimer la colonne 'CP' originale
df_model = df_model.drop(columns=['CodePostal', 'TypeLocalName'])

In [9]:
df_model

,Prix,SurfaceTerrain,SurfaceBati,SurfaceCarrez,NombreLots,NombrePiecesPrincipales,CodePostal_33000,CodePostal_33100,CodePostal_33110,CodePostal_33112,...,CodePostal_33890,CodePostal_33910,CodePostal_33920,CodePostal_33930,CodePostal_33950,CodePostal_33970,CodePostal_33980,CodePostal_33990,TypeLocalName_Appartement,TypeLocalName_Maison
0,103950,0,19,19,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,214380,0,38,39,2,1,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,177000,0,63,0,3,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,153000,0,30,0,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,223170,0,61,0,3,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147835,140000,1922,200,0,8,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
147836,80000,237,73,0,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
147837,155000,600,78,0,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
147838,111150,51,82,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


# SEPARATION DU JEU DE DONNES

In [10]:
# Séparation du jeu de données en X (caractéristiques) et y (cible)
X, y = df_model.drop(columns=["Prix"]), df_model["Prix"]

# Séparation en ensembles de train, de test, et de validation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.10, random_state=42)

In [11]:
# Instancier le RobustScaler
scaler = RobustScaler()

# Sélectionner les colonnes numériques dans X_train pour l'application du RobustScaler
num_cols = X_train.select_dtypes(include=['number']).columns

# Appliquer le RobustScaler uniquement sur les colonnes numériques de l'ensemble d'entraînement
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])

# Utiliser le scaler ajusté pour transformer les ensembles de test et de validation
X_test[num_cols] = scaler.transform(X_test[num_cols])
X_val[num_cols] = scaler.transform(X_val[num_cols])

print(f'X_train : {X_train.shape[0]}, X_test : {X_test.shape[0]}, X_val : {X_val.shape[0]}')

X_train : 99052, X_test : 43909, X_val : 4879


In [12]:
# Étape 1: Instancier le modèle
rf = RandomForestRegressor(n_estimators=100, random_state=42)

In [13]:
# Étape 2: Entraîner le modèle
rf.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [14]:
# Étape 3: Faire des prédictions sur l'ensemble de test
y_pred = rf.predict(X_test)

In [15]:
# Étape 4: Évaluer les performances
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R-squared: {round(r2*100)} % du prix est expliquer par nos features actuelles")

Mean Squared Error: 12562013799.345013
Mean Absolute Error: 68243.05876197164
R-squared: 66 % du prix est expliquer par nos features actuelles
